In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime

In [2]:
df = pd.read_csv("../data/mtd_combined.csv", index_col=0)
df.head(2)

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,...,rating_other_comments,cancellation_reason,cancellation_comment,sheet_name,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two
0,5727475e-8224-4302-9228-c92b9d4a5220,f8ff0526-887a-4e48-ad96-977e12fd70c1,5483,1.0,4.65,4.65,0.0,0.0,STANDARD,Würzmühle,...,NaN,NaN,NaN,MTD,NaN,NaN,NaN,NaN,NaN,NaN
1,18fec0a6-b7ba-442b-8472-04bdb6ba1b86,51e1a1a8-995c-488c-84ce-3789e46f0417,3575,1.0,0.00,2.77,0.0,0.0,BAHN_CARD,Julius-Wilde-Straße,...,NaN,NaN,NaN,MTD,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df[df['id']=='6de9cc6f-fe92-4964-a400-53491de46155']['pickup_arrival_time']

1568    1899-12-29 23:58:42
Name: pickup_arrival_time, dtype: object

In [3]:
df_stops = pd.read_excel("../data/MoDstops+Preismodell.xlsx", sheet_name="MoDstops")
df_stops.head(2)

,MoDStop Id,MoDStop Name,MoDStop Lat,MoDStop Long,MoDStop Adresse
0,1001,Mandelgasse,49.351780,8.129000,"Seilerbahn 1, 67433 Neustadt"
1,1002,Hauptfeuerwache,49.353733,8.131552,"Lindenstraße 11, 67433 Neustadt"


#### Column: id

In [4]:
print("# of NaN:", int(df[['id']].isna().sum()))
print("# of duplicates:", df.id.duplicated().sum() - int(df[['id']].isna().sum() - 1), "(exklusive NaN)")

# of NaN: 4535
# of duplicates: 0 (exklusive NaN)


In [5]:
df.loc[(df['id'].duplicated()) & (~df['id'].isna()), :]

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,...,rating_other_comments,cancellation_reason,cancellation_comment,sheet_name,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two


Wieso gibts doppelte ids? (nur bei combined rides, nicht bei combined mtd)

Fehlende ID's sind problematisch, wenn wir die ride id als Primärschlüssel nutzen wollen. Es handelt sich um rides, bei denen der Status 'offer' oder 'offer_rejected' ist. 

#### Column: user_id

In [6]:
print("# of NaN:", int(df[['user_id']].isna().sum()))
print("# of duplicates:", df.user_id.duplicated().sum() - int(df[['user_id']].isna().sum()), "(exklusive NaN)")

# of NaN: 0
# of duplicates: 9565 (exklusive NaN)


#### Column: distance

In [7]:
print("# of NaN:", int(df[['distance']].isna().sum()))
print("# of neg:", len(df.loc[(df.distance < 0)]))
print("# of zero:", len(df.loc[(df.distance == 0)]))
print("# of ones:", len(df.loc[(df.distance == 1)]))

# of NaN: 0
# of neg: 0
# of zero: 0
# of ones: 14


In [8]:
comparison = np.where(df['pickup_address'] == df['dropoff_address'], True, False)
df.loc[:, 'isSameAddress'] = comparison
mask = df['distance']==1
df[mask][['pickup_address', 'dropoff_address', 'isSameAddress']]

,pickup_address,dropoff_address,isSameAddress
37,49.322557|8.12336,49.322557|8.12336,True
49,Haardter Winzer,Haardter Winzer,True
114,Am Herzel West,Am Herzel West,True
418,TC Rot-Weiß,TC Rot-Weiß,True
646,Schütt,Schütt,True
707,49.330245|8.130933,49.330245|8.130933,True
841,49.351731|8.136637,49.351731|8.136637,True
972,49.376347|8.153444,49.376347|8.153444,True
1047,49.316193|8.135477,49.316193|8.135477,True
1164,Schütt,Schütt,True


Was sollen wir mit distance=1m und gleichem Start- & Endhaltestopp machen? 

#### Column: number_of_passenger

In [9]:
print("# of NaN:", int(df[['number_of_passenger']].isna().sum()))
print("# of neg:", len(df.loc[(df.number_of_passenger < 0)]))
print("# of zero:", len(df.loc[(df.number_of_passenger == 0)]))
print("# of pos:", len(df.loc[(df.number_of_passenger > 0)]))
print('Max # of passengers:', df['number_of_passenger'].max())

# of NaN: 4535
# of neg: 0
# of zero: 0
# of pos: 5813
Max # of passengers: 5.0


Fehlende Angaben bei number_of_passenger: Es handelt sich um rides, bei denen der Status 'offer' oder 'offer_rejected' ist. 
--> Bei Analysen vermutlich sowieso filtern auf completed

#### Column: price_operations

In [10]:
print("# of NaN:", int(df[['price_operations']].isna().sum()))
print("# of neg:", len(df.loc[(df.price_operations < 0)]))
print("# of zero:", len(df.loc[(df.price_operations == 0)]))
print("# of pos:", len(df.loc[(df.price_operations > 0)]))
print('Min price:', df['price_operations'].min())
print('Max price:', df['price_operations'].max())

# of NaN: 1204
# of neg: 0
# of zero: 458
# of pos: 8686
Min price: 0.0
Max price: 33.1


#### Column: price_offer

In [11]:
print("# of NaN:", int(df[['price_offer']].isna().sum()))
print("# of neg:", len(df.loc[(df.price_offer < 0)]))
print("# of zero:", len(df.loc[(df.price_offer == 0)]))
print("# of pos:", len(df.loc[(df.price_offer > 0)]))
print('Min price:', df['price_offer'].min())
print('Max price:', df['price_offer'].max())

# of NaN: 4535
# of neg: 0
# of zero: 0
# of pos: 5813
Min price: 0.8
Max price: 32.3


#### Column: price_payed

In [12]:
print("# of NaN:", int(df[['price_payed']].isna().sum()))
print("# of neg:", len(df.loc[(df.price_payed < 0)]))
print("# of zero:", len(df.loc[(df.price_payed == 0)]))
print("# of pos:", len(df.loc[(df.price_payed > 0)]))
print('Min price:', df['price_payed'].min())
print('Max price:', df['price_payed'].max())

# of NaN: 4535
# of neg: 0
# of zero: 1322
# of pos: 4491
Min price: 0.0
Max price: 555.33


#### Column: free_ride

In [13]:
print("# of NaN:", int(df[['free_ride']].isna().sum()))


# of NaN: 4535


Dies sind genau die rides, die keine ride id haben

In [14]:
df.free_ride.value_counts()

0.0    5170
1.0     643
Name: free_ride, dtype: int64

#### Column: pickup_address and dropoff_address

In [15]:
print("# of NaN:", int(df[['pickup_address']].isna().sum()))
print("# of NaN:", int(df[['dropoff_address']].isna().sum()))


# of NaN: 0
# of NaN: 0


In [16]:
df['start_id'] = ""
df['end_id'] = ""

def get_stop_id(address):
    if address[0].isdigit():
        lat = address.split("|")[0]
        long = address.split("|")[1]
        for index, row in df_stops.iterrows():
            if str(row["MoDStop Lat"]) == lat and str(row["MoDStop Long"]) == long:
                return row["MoDStop Id"]
        return "No match of lat and long"
    else:
        # different naming
        if address == "Rewe Mußbach":
            address = address + " (Shoppenwiese)"
        for index, row in df_stops.iterrows():
            if row["MoDStop Name"] == address:
                return row["MoDStop Id"]
        return "No match of address name"
            		
        

In [17]:
for index, row in df.iterrows():
    df.at[index, 'start_id'] = get_stop_id(row['pickup_address'])
    df.at[index, 'end_id'] = get_stop_id(row['dropoff_address'])

In [18]:
df.to_excel("test.xlsx")

#### Column: State

In [19]:
print("# of NaN:", int(df[['state']].isna().sum()))

# of NaN: 0


In [20]:
df.state.value_counts()

completed         4862
offer             4365
canceled           857
offer-rejected     170
rejected            93
active               1
Name: state, dtype: int64

Was bedeutet ‚active‘, ‚rejected‘? Und ist offer-rejected wie cancelled zu betrachten?

#### Column: Created_from_offer

In [21]:
print("# of NaN:", int(df[['created_from_offer']].isna().sum()))

# of NaN: 3065


#### Column: created_at

In [22]:
print("# of NaN:", int(df[['created_at']].isna().sum()))

# of NaN: 0


#### Column: scheduled_to

In [23]:
print("# of NaN:", int(df[['scheduled_to']].isna().sum()))

# of NaN: 7770


In [36]:
comparison = np.where(df['created_at'] == df['dispatched_at'], True, False)
df.loc[:, 'isSame'] = comparison
mask = df['scheduled_to'].isna()
df_temp = df[mask]
df_temp[:][['created_at', 'dispatched_at', 'isSame']]

,created_at,dispatched_at,isSame
2,2021-07-01 09:21:39,2021-07-01 09:21:39,True
3,2021-07-01 11:24:01,2021-07-01 11:24:01,True
4,2021-07-01 13:12:19,2021-07-01 13:12:19,True
5,2021-07-01 16:37:49,2021-07-01 16:37:49,True
6,2021-07-01 17:11:11,2021-07-01 17:11:11,True
...,...,...,...
10342,2022-02-28 22:44:38,2022-02-28 22:44:38,True
10343,2022-02-28 22:45:35,2022-02-28 22:45:35,True
10344,2022-02-28 22:48:07,2022-02-28 22:48:07,True
10345,2022-02-28 22:50:10,2022-02-28 22:50:10,True


In [37]:
df_temp[df['isSame']==False]

C:\Users\janpa\AppData\Local\Temp\ipykernel_23308\2517812279.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp[df['isSame']==False]


,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,...,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,isSameAddress,start_id,end_id,isSame
603,3ce77825-b758-426d-8595-48cda832f801,76f7bb28-ae72-4d6b-83f5-ce252a641f81,2983,1.0,3.4,3.4,3.4,0.0,STANDARD,Alter Turnplatz,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1007,11003,False
605,NaN,8d46a720-a53a-44b2-99f8-138823ea42e0,1855,NaN,NaN,NaN,NaN,NaN,STANDARD,49.336132|8.130418,...,NaN,NaN,NaN,NaN,NaN,NaN,False,4041,3006,False
606,279824bc-f10e-4854-a9bb-acebe8bccafd,ea92845e-9a09-42f7-92cb-52e139479ba2,7306,1.0,8.6,8.6,0.0,1.0,STANDARD,Mandelring,...,NaN,NaN,NaN,NaN,NaN,NaN,False,12007,4025,False
607,NaN,106ec942-75cb-4ba5-8371-6f4404571f39,3978,NaN,NaN,NaN,NaN,NaN,VRN,49.325872|8.11613,...,NaN,NaN,NaN,NaN,NaN,NaN,False,4029,3013,False
608,8e5f4295-6701-4ce4-b805-f27af9fb66a8,44f61d06-8e79-42c6-9abd-0e85fcaf9d6d,5926,2.0,1.9,4.6,0.0,1.0,VRN,Karolinenstraße,...,NaN,NaN,NaN,NaN,NaN,NaN,False,3001,4029,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3628,NaN,b6537f08-519c-4c67-ad43-6996aca5fb9a,6150,NaN,NaN,NaN,NaN,NaN,BAHN_CARD,49.368881|8.14174,...,NaN,NaN,NaN,NaN,NaN,NaN,False,12007,2007,False
3629,f72309ab-151f-42fb-a98e-cba05a84cb2b,76f7bb28-ae72-4d6b-83f5-ce252a641f81,2314,1.0,3.1,3.1,3.1,0.0,STANDARD,Schütt,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1009,11003,False
3630,NaN,76f7bb28-ae72-4d6b-83f5-ce252a641f81,2845,NaN,NaN,NaN,NaN,NaN,STANDARD,49.356253|8.108031,...,NaN,NaN,NaN,NaN,NaN,NaN,False,11003,1009,False
3631,NaN,1a6b4e7f-6766-4d39-beef-9df24e641128,4003,NaN,NaN,NaN,NaN,NaN,STANDARD,49.328803|8.130412,...,NaN,NaN,NaN,NaN,NaN,NaN,False,4025,1009,False


In [34]:
df.isSame.value_counts()

True     6098
False    4250
Name: isSame, dtype: int64

#### Column: dispatched_at

In [43]:
print("# of NaN:", int(df[['dispatched_at']].isna().sum()))

# of NaN: 32


In [41]:
comparison = np.where(df['created_at'] <= df['dispatched_at'], True, False)
df.loc[:, 'created_at_before_dispatched_at'] = comparison
df[df['created_at_before_dispatched_at']==False]

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,...,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,isSameAddress,start_id,end_id,isSame,created_at_before_dispatched_at
603,3ce77825-b758-426d-8595-48cda832f801,76f7bb28-ae72-4d6b-83f5-ce252a641f81,2983,1.0,3.40,3.40,3.40,0.0,STANDARD,Alter Turnplatz,...,NaN,NaN,NaN,NaN,NaN,False,1007,11003,False,False
604,5d7a3b12-bf3c-4615-a9dc-637c6efc81c4,f07028da-ca7e-4713-9e45-743c71712e80,3328,1.0,3.45,1.55,1.55,0.0,VRN,Jüdischer Friedhof,...,NaN,NaN,NaN,NaN,NaN,False,10003,6004,False,False
605,NaN,8d46a720-a53a-44b2-99f8-138823ea42e0,1855,NaN,NaN,NaN,NaN,NaN,STANDARD,49.336132|8.130418,...,NaN,NaN,NaN,NaN,NaN,False,4041,3006,False,False
606,279824bc-f10e-4854-a9bb-acebe8bccafd,ea92845e-9a09-42f7-92cb-52e139479ba2,7306,1.0,8.60,8.60,0.00,1.0,STANDARD,Mandelring,...,NaN,NaN,NaN,NaN,NaN,False,12007,4025,False,False
607,NaN,106ec942-75cb-4ba5-8371-6f4404571f39,3978,NaN,NaN,NaN,NaN,NaN,VRN,49.325872|8.11613,...,NaN,NaN,NaN,NaN,NaN,False,4029,3013,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5356,65841607-adab-490e-9376-1b917c646a0f,4d428728-0aca-4345-aee1-022366c189f9,8883,2.0,10.80,10.80,10.80,0.0,STANDARD,Hauberallee,...,NaN,NaN,NaN,NaN,NaN,False,2002,14003,False,False
7288,7a88f7ad-e1c4-4ba4-90e7-4f004f0c24e1,4a35f2cd-c7df-43cc-b15b-7e2cd32473c7,2880,1.0,3.60,3.60,0.00,0.0,STANDARD,Oswald-Wiersich-Straße,...,NaN,NaN,NaN,NaN,NaN,False,9017,9005,False,False
7495,f1a7f16a-554c-4d2b-9293-4ab894bdb932,8c5545e5-72cd-4163-80bf-8d975695642f,4800,1.0,4.64,2.49,2.49,0.0,VRN,Hauptbahnhof,...,NaN,NaN,NaN,NaN,NaN,False,1008,4025,False,False
7973,e47cc4c0-d2e3-42cb-b66e-6241ce19e450,63adf4da-26c9-4ab9-baac-583aa1166f04,3933,1.0,4.20,4.20,4.20,0.0,STANDARD,Rathaus,...,NaN,NaN,NaN,NaN,NaN,False,1005,15002,False,False


#### Column: pickup_arrival_time

In [45]:
print("# of NaN:", int(df[['pickup_arrival_time']].isna().sum()))

# of NaN: 5582


#### Column: arriving_push

In [46]:
print("# of NaN:", int(df[['arriving_push']].isna().sum()))

# of NaN: 5435


#### Column: vehicle_arrived_at

In [47]:
print("# of NaN:", int(df[['vehicle_arrived_at']].isna().sum()))

# of NaN: 5582


#### Column: earliest_pickup_expectations

#### Column: pickup_first_eta, pickup_eta, pickup_at

In [49]:
print("# of NaN:", int(df[['pickup_at']].isna().sum()))

# of NaN: 5496


#### Column: dropoff_first_eta, dropoff_eta, dropoff_at

In [48]:
print("# of NaN:", int(df[['dropoff_at']].isna().sum()))

# of NaN: 5490


#### Column: updated_at

#### Column: arrival_deviation, waiting_time, boarding_time, ride_time, trip_time, shortest_ridetime, delay


In [50]:
print("arrival_deviation")
print("# of NaN:", int(df[['arrival_deviation']].isna().sum()))
print("# of neg:", len(df.loc[(df.arrival_deviation < 0)]))
print("# of zero:", len(df.loc[(df.arrival_deviation == 0)]))
print("# of pos:", len(df.loc[(df.arrival_deviation > 0)]))
print('Min:', df['arrival_deviation'].min())
print('Max:', df['arrival_deviation'].max())

arrival_deviation
# of NaN: 6082
# of neg: 2007
# of zero: 439
# of pos: 1820
Min: -2535490.0
Max: 2445838.0


In [51]:
print("waiting_time")
print("# of NaN:", int(df[['waiting_time']].isna().sum()))
print("# of neg:", len(df.loc[(df.waiting_time < 0)]))
print("# of zero:", len(df.loc[(df.waiting_time == 0)]))
print("# of pos:", len(df.loc[(df.waiting_time > 0)]))
print('Min:', df['waiting_time'].min())
print('Max:', df['waiting_time'].max())

waiting_time
# of NaN: 5599


TypeError: '<' not supported between instances of 'str' and 'int'

#### Column: longer_route_factor, arrival_indicator

#### Column: rating, rating_X

#### Column: cancellation_reason, cancellation_comment, cancelled_at


#### Column: bahn_card_number, year_card_type, year_card_number